In [ ]:
%%capture
!pip install sentence_transformers

!pip install transformers

!pip install sentencepiece

!pip install nlpaug

#!pip install sacremoses #for backtranslate

In [ ]:
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
strategy = tf.distribute.get_strategy()
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  import sys


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Sentence Correlation/train.xlsx")
test = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Sentence Correlation/evaluation.xlsx")

In [ ]:
len(test)

9000

In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 50#32
epochs = 12

# Labels in our dataset.
#labels = ["contradiction", "entailment", "neutral"]

## Load the Data

In [ ]:
# There are more than 550k samples in total; we will use 100k for this example.
'''train_df = pd.read_csv("SNLI_Corpus/snli_1.0_train.csv", nrows=100000)
valid_df = pd.read_csv("SNLI_Corpus/snli_1.0_dev.csv")
test_df = pd.read_csv("SNLI_Corpus/snli_1.0_test.csv")

# Shape of the data
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {valid_df.shape[0]}")
print(f"Total test samples: {valid_df.shape[0]}")'''

'train_df = pd.read_csv("SNLI_Corpus/snli_1.0_train.csv", nrows=100000)\nvalid_df = pd.read_csv("SNLI_Corpus/snli_1.0_dev.csv")\ntest_df = pd.read_csv("SNLI_Corpus/snli_1.0_test.csv")\n\n# Shape of the data\nprint(f"Total train samples : {train_df.shape[0]}")\nprint(f"Total validation samples: {valid_df.shape[0]}")\nprint(f"Total test samples: {valid_df.shape[0]}")'

In [ ]:
'''print(f"Sentence1: {train_df.loc[1, 'sentence1']}")
print(f"Sentence2: {train_df.loc[1, 'sentence2']}")
print(f"Similarity: {train_df.loc[1, 'similarity']}")'''

'print(f"Sentence1: {train_df.loc[1, \'sentence1\']}")\nprint(f"Sentence2: {train_df.loc[1, \'sentence2\']}")\nprint(f"Similarity: {train_df.loc[1, \'similarity\']}")'

# Data Augmentation

## Using a BERT Base model to find the most semantically similar reasons to a text and using them as a reason in a new sample.

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
from sklearn.metrics.pairwise import cosine_similarity

text_embeddings = model.encode(train['text'])
reason_embeddings = model.encode(train['reason'])

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
'''
train2 = pd.DataFrame(columns=["text", "reason", "label"])
for i in range(len(train)):
    try: text_sim_with_all_reasons = cosine_similarity([text_embeddings[i]],
                                                    reason_embeddings)[0]
    except IndexError: pass    
    highest_sim_indices = np.argsort(text_sim_with_all_reasons)[-3:-1]

    train2.reset_index(inplace=True, drop=True)
    for j in range(2):
        a = [train['text'][i], train['reason'][highest_sim_indices[j]], 1.0]
        train2 = train2.append(pd.DataFrame([a], columns=["text", "reason", "label"]), ignore_index=True)'''

'\ntrain2 = pd.DataFrame(columns=["text", "reason", "label"])\nfor i in range(len(train)):\n    try: text_sim_with_all_reasons = cosine_similarity([text_embeddings[i]],\n                                                    reason_embeddings)[0]\n    except IndexError: pass    \n    highest_sim_indices = np.argsort(text_sim_with_all_reasons)[-3:-1]\n\n    train2.reset_index(inplace=True, drop=True)\n    for j in range(2):\n        a = [train[\'text\'][i], train[\'reason\'][highest_sim_indices[j]], 1.0]\n        train2 = train2.append(pd.DataFrame([a], columns=["text", "reason", "label"]), ignore_index=True)'

In [ ]:
#train = pd.concat([train, train2], axis=0)
train.dropna(inplace=True)

## Randomly assigning a reason to each text and setting the label to 0


In [ ]:
import random
train2 = pd.DataFrame()
train2['text'] = train['text']
a = list(train['reason'])
random.shuffle(a)
train2['reason'] = a
train2['label'] = [0.0]*len(train2)
train2

,text,reason,label
0,this is an amazing app for online classes!but,zoom is not convenient,0.0
1,very practical and easy to use,app developers need to turn on sound when logg...,0.0
2,this app is very good for video conferencing.,want to load profile page from hamburger menu,0.0
3,i can not download this zoom app,unable to open app on pixel device,0.0
4,i am not able to download this app,web browser audio not working,0.0
...,...,...,...
2056,i am not getting jio tv properly.,good for video conferencing,0.0
2057,(learn from the netflix interface),want to connect sound to bugles via bluetooth,0.0
2058,"hello,\ndisney+ must also be installed on chro...",different prices come every month,0.0
2059,it is a shame that disney+ does not work on tv...,want double speed function,0.0


In [ ]:
train = pd.concat([train, train2], axis=0)

In [ ]:
validation = train.sample(frac =.2)
train = train.sample(frac =.8)

## Other data augmentation tools: Replacing words with synonyms and antonyms from wordnet, and using a DistilBERT to paraphrase texts and reasons.

In [ ]:
import nlpaug.augmenter.word as naw
synaug = naw.SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=1, aug_max=10, aug_p=0.3, lang='eng', 
                        stopwords=None, tokenizer=None, reverse_tokenizer=None, stopwords_regex=None, force_reload=False, 
                        verbose=0)

ant_aug = naw.AntonymAug(name='Antonym_Aug', aug_min=1, aug_max=10, aug_p=0.3, lang='eng', stopwords=None, tokenizer=None, 
                        reverse_tokenizer=None, stopwords_regex=None, verbose=0)

aug_bert = naw.ContextualWordEmbsAug(
        model_path='distilbert-base-uncased', 
        device='cuda',
        action='insert', top_k=20)

#back_translation_aug = naw.BackTranslationAug() 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def synonymaug(text):
    return synaug.augment(text)[0]

def antonymaug(text):
    return ant_aug.augment(text)[0]

def flowaug(text):
    return aug_bert.augment(text)[0]

def backtranslate(tex):
    return back_translation_aug.augment(text)[0]

'''def word2vecaug(text):
    aug_w2v = naw.WordEmbsAug(
        model_type='glove', model_path='/content/glove.6B.300d.txt',
        action="substitute")
    return aug_w2v.augment(text)'''

'def word2vecaug(text):\n    aug_w2v = naw.WordEmbsAug(\n        model_type=\'glove\', model_path=\'/content/glove.6B.300d.txt\',\n        action="substitute")\n    return aug_w2v.augment(text)'

In [ ]:
def applyaugtodf(df, augfunction):
    train2 = pd.DataFrame()
    train2['text'] = train['text'].progress_apply(augfunction)
    train2['reason'] = train['reason'].progress_apply(augfunction)
    train2['label'] = train['label']
    return train2

In [ ]:
#train = pd.concat([train, applyaugtodf(train, synonymaug), applyaugtodf(train, antonymaug)], axis=0)

In [ ]:
print(len(train))
train.drop_duplicates(inplace=True)
print(len(train))

train.to_csv("/content/drive/MyDrive/train_augmented.csv", index=False)
train

3298
3298


,text,reason,label
966,break room feature is not available on mobile ...,want to register for eucharist,0.0
1357,"because of this app, i can not communicate wit...",want to turn on orientation lock,0.0
1458,"i like it is great and very easy to handle, bo...",want to extend meeting timelimit,0.0
1058,but i problem is the video cannot be paused be...,want to pause video,1.0
974,please add free resources for teachers,phone keeps crashing,0.0
...,...,...,...
1641,love the app as i get to keep up with all spor...,good for sports,1.0
827,call gets disconnected and app restarts,call gets disconnected and app restarts,1.0
1570,best ott app in the world,want to schedule a meeting through mobile and ...,0.0
1928,I am using 150mbps but the streaming quality i...,android tv version is bad,1.0


In [ ]:
#train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sentence Correlation/train_augmented.csv")

In [ ]:
train_df = train
'''(
    #train_df[train_df.similarity != "-"]
    train
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)'''

valid_df = validation
'''(
    #valid_df[valid_df.similarity != "-"]
    test
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)'''
test_df = test
'''(
    #train_df[train_df.similarity != "-"]
    test
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)'''
len(test_df)

9000

In [ ]:
'''train_df["label"] = train_df["similarity"].apply(
    lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2
)'''
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=2)
'''
valid_df["label"] = valid_df["similarity"].apply(
    lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2
)'''
y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=2)
'''
test_df["label"] = test_df["similarity"].apply(
    lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2
)'''
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=2)

In [ ]:
len(y_test)

9000

## Model Definition from Keras Docs

In [ ]:

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


## Build the model.

In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f72097f37d0>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',        

Create train and validation data generators

In [ ]:
train_data = BertSemanticDataGenerator(
    train_df[["text", "reason"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)

valid_data = BertSemanticDataGenerator(
    valid_df[["text", "reason"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

## Model 1


In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/24
65/65 [==============================] - ETA: 0s - loss: 0.4339 - acc: 0.7969

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


65/65 [==============================] - 58s 610ms/step - loss: 0.4339 - acc: 0.7969 - val_loss: 0.1248 - val_acc: 0.9650
Epoch 2/24
65/65 [==============================] - 36s 549ms/step - loss: 0.1498 - acc: 0.9480 - val_loss: 0.0825 - val_acc: 0.9762
Epoch 3/24
65/65 [==============================] - 36s 555ms/step - loss: 0.1359 - acc: 0.9529 - val_loss: 0.0745 - val_acc: 0.9800
Epoch 4/24
65/65 [==============================] - 37s 564ms/step - loss: 0.1157 - acc: 0.9557 - val_loss: 0.0611 - val_acc: 0.9825
Epoch 5/24
65/65 [==============================] - 37s 568ms/step - loss: 0.1093 - acc: 0.9594 - val_loss: 0.0562 - val_acc: 0.9850
Epoch 6/24
65/65 [==============================] - 37s 574ms/step - loss: 0.1030 - acc: 0.9606 - val_loss: 0.0472 - val_acc: 0.9862
Epoch 7/24
65/65 [==============================] - 37s 578ms/step - loss: 0.0921 - acc: 0.9689 - val_loss: 0.0506 - val_acc: 0.9825
Epoch 8/24
65/65 [==============================] - 37s 577ms/step - loss: 0.085

In [ ]:
len(test_df)

9000

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[["text", "reason"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
#model.evaluate(test_data, verbose=1)

In [ ]:
len(test_data)

180

In [ ]:
model1pred = model.predict(test_data)#, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will alwa

In [ ]:
len(test_data)

180

In [ ]:
model1pred = [list(i).index(max(list(i))) for i in list(model1pred)]
len(model1pred)

9000

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def givemetrics(true, pred):
    print(accuracy_score(true, pred), recall_score(true, pred), precision_score(true, pred), f1_score(true, pred))

In [ ]:
givemetrics(model1pred, test['label'])

0.57 0.43295787687085324 0.935021659446851 0.5918582577515291


In [ ]:
test

,text,reason,label
0,the app is crashing when i play a vedio,app crashes during playback,1.0
1,but i want to connect it to the tv from one de...,want compatibility with more smart televisions,0.0
2,very helpful when and home working remotley,good app for work,0.0
3,this zoom so called and missed call and mobile...,receiving incorrect phone number message,0.0
4,one of my favorite apps,good for spending time,0.0
...,...,...,...
8995,very good streaming platform has a lot of content,app has very good content,1.0
8996,so paying for an app that is not fit for purpose.,unable to use app,0.0
8997,"it does not work well for me, it loads, but if...",app keeps loading indefinitely,0.0
8998,i can not watch the new disney shows.,unable to load the episodes list,0.0


## Model 2

In [ ]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_masks[0][0]',    

In [ ]:
history = model.fit(
    train_data,
    #validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Epoch 1/12
65/65 [==============================] - 81s 1s/step - loss: 0.0176 - accuracy: 0.9942
Epoch 2/12
65/65 [==============================] - 82s 1s/step - loss: 0.0089 - accuracy: 0.9978
Epoch 3/12
65/65 [==============================] - 82s 1s/step - loss: 0.0103 - accuracy: 0.9969
Epoch 4/12
65/65 [==============================] - 82s 1s/step - loss: 0.0056 - accuracy: 0.9991
Epoch 5/12
65/65 [==============================] - 82s 1s/step - loss: 0.0024 - accuracy: 0.9997
Epoch 6/12
65/65 [==============================] - 82s 1s/step - loss: 0.0021 - accuracy: 0.9997
Epoch 7/12
65/65 [==============================] - 82s 1s/step - loss: 0.0021 - accuracy: 0.9994
Epoch 8/12
65/65 [==============================] - 82s 1s/step - loss: 0.0024 - accuracy: 0.9994
Epoch 9/12
65/65 [==============================] - 82s 1s/step - loss: 0.0026 - accuracy: 0.9991
Epoch 10/12
65/65 [==============================] - 82s 1s/step - loss: 0.0025 - accuracy: 0.9997
Epoch 11/12
65/65 [

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[["text", "reason"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
#model.evaluate(test_data)#, verbose=1)

In [ ]:
model2pred = model.predict(test_data)#, verbose=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will alwa

In [ ]:
model2pred = [list(i).index(max(list(i))) for i in list(model2pred)]

In [ ]:
givemetrics(model2pred, test['label'])

0.5406666666666666 0.4179580014482259 0.9616794401866045 0.5826771653543307


## Model 3

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
from sklearn.metrics.pairwise import cosine_similarity

text_embeddings = model.encode(list(train['text']))
reason_embeddings = model.encode(list(train['reason']))

In [ ]:
predx = [cosine_similarity([text_embeddings[i]], [reason_embeddings[i]])[0] for i in range(len(text_embeddings))]
for j in [0.4, 0.425, 0.45, 0.475, 0.5, 0.525, 0.55, 0.575, 0.6, 0.625, 0.65, 0.675]:
    print(j)
    preds = [1 if i > j else 0 for i in predx]
    givemetrics(preds, train['label'])
    print("------------------------------------------")

0.4
0.7246816252274105 0.6832116788321168 0.8452739313666466 0.7556512378902045
------------------------------------------
0.425
0.7355973317161917 0.7036654620547238 0.8205900060204696 0.7576431350750417
------------------------------------------
0.45
0.7419648271679806 0.7217962760131434 0.7934978928356412 0.7559506739317465
------------------------------------------
0.475
0.7459066100667071 0.738827626233314 0.7664057796508128 0.7523640661938534
------------------------------------------
0.5
0.750758035172832 0.7578365089121082 0.7423239012642986 0.75
------------------------------------------
0.525
0.7598544572468162 0.7864205669083718 0.7182420228777845 0.7507866582756451
------------------------------------------
0.55
0.7631898120072771 0.8094233473980309 0.6929560505719446 0.7466753162504054
------------------------------------------
0.575
0.7577319587628866 0.81973293768546 0.6652618904274533 0.7344632768361582
------------------------------------------
0.6
0.7592480291085506 0

In [ ]:
text_embeddings = model.encode(list(test['text']))
reason_embeddings = model.encode(list(test['reason']))

In [ ]:
predx = [cosine_similarity([text_embeddings[i]], [reason_embeddings[i]])[0] for i in range(len(text_embeddings))]
for j in [0.4, 0.425, 0.45, 0.475, 0.5, 0.525, 0.55, 0.575, 0.6, 0.625, 0.65, 0.675]:
    print(j)
    preds = [1 if i > j else 0 for i in predx]
    givemetrics(preds, test['label'])
    print("------------------------------------------")

0.4
0.5192222222222223 0.40385730858468677 0.9280239920026658 0.5627968071132666
------------------------------------------
0.425
0.5421111111111111 0.4149194773625038 0.9100299900033322 0.5699676510487321
------------------------------------------
0.45
0.5685555555555556 0.42921348314606744 0.8910363212262579 0.5793521828620951
------------------------------------------
0.475
0.5881111111111111 0.43990466462376576 0.8610463178940353 0.5823098591549295
------------------------------------------
0.5
0.6095555555555555 0.4535073409461664 0.8337220926357881 0.5874618455036393
------------------------------------------
0.525
0.6274444444444445 0.4658782473827065 0.8007330889703432 0.5890427748498591
------------------------------------------
0.55
0.6472222222222223 0.48172268907563026 0.7640786404531823 0.5909032341193146
------------------------------------------
0.575
0.663 0.49635036496350365 0.7250916361212929 0.5893026404874746
------------------------------------------
0.6
0.67822222